# Fact checking

In [311]:
import os
import requests
import zipfile
import re
import string
import random
from collections import namedtuple
from functools import partial

import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from tqdm.notebook import tqdm

import utils

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [109]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

## Dataset pre-processing

In [110]:
def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)


def download_data(data_path):
    toy_data_path = os.path.join(data_path, "fever_data.zip")
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(
                toy_url, params={"id": toy_data_url_id}, stream=True
            )
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

In [111]:
dataset_folder = os.path.join("datasets", "fever")
download_data(dataset_folder)

## Dataset conversion

In [224]:
dfs = []
for split in ["train", "test", "val"]:
    split_file = os.path.join(dataset_folder, f"{split}_pairs.csv")
    if os.path.isfile(split_file):
        split_df = pd.read_csv(split_file, index_col=0)
        split_df["split"] = pd.Series([split] * len(split_df), index=split_df.index)
        dfs.append(split_df)

df = pd.concat(dfs)
df.columns = map(str.lower, df.columns)
df = df.reset_index(drop=True)
df.head()

,claim,evidence,id,label,split
0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS,train
1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS,train
2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES,train
3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS,train
4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES,train


In [113]:
df["label"].unique()

array(['SUPPORTS', 'REFUTES'], dtype=object)

In [217]:
print(f'Training set shape: {df[df["split"] == "train"].shape}')
print(f'Validation set shape: {df[df["split"] == "val"].shape}')
print(f'Test set shape: {df[df["split"] == "test"].shape}')

Training set shape: (121740, 6)
Validation set shape: (7165, 6)
Test set shape: (7189, 6)


In [207]:
TAB_CHAR = "\t"
PERIOD_CHAR = "."
LEFT_PARENS = "-LRB-"
RIGHT_PARENS = "-RRB-"
SQUARE_BRACKETS = r"-LSB-.*-RSB-"


def preprocess_claim(text):
    if text[-1] in string.punctuation:
        text = text[:-1]
    return text.strip().lower()


def preprocess_evidence(text):
    # Remove everything before the first tab character
    text = text[text.find(TAB_CHAR) + len(TAB_CHAR) :]
    # Replace tabs with spaces
    text = text.replace(TAB_CHAR, " ")
    # Remove parenthesis
    text = text.replace(LEFT_PARENS, " ").replace(RIGHT_PARENS, " ")
    # Remove everything between square brackets
    text = re.sub(SQUARE_BRACKETS, "", text)
    # Remove everything after the last period
    last_period = text.rfind(PERIOD_CHAR)
    if last_period is not None:
        text = text[:last_period]
    # Remove punctuation
    text = text.translate(text.maketrans("", "", string.punctuation))
    # Remove extra whitespaces
    text = " ".join(text.split())
    # Convert to lowercase
    return text.lower()

In [210]:
random_index = random.choice(df.index.tolist())
random_evidence = df.loc[random_index]["evidence"]
preprocessed_random_evidence = preprocess_evidence(random_evidence)
print(f"Random evidence {random_index}")
print(f"{random_evidence}")
print(f"{preprocessed_random_evidence}")

Random evidence 89133
1	He won an Academy Award for Best Actor , BAFTA Award for Best Actor in a Leading Role , and Golden Globe Award for Best Actor in a Musical or Comedy , for his portrayal of Ray Charles in the 2004 biographical film Ray .	Academy Award for Best Actor	Academy Award for Best Actor	BAFTA Award	BAFTA Award	Golden Globe Award	Golden Globe Award	Ray Charles	Ray Charles	Ray	Ray (film)
he won an academy award for best actor bafta award for best actor in a leading role and golden globe award for best actor in a musical or comedy for his portrayal of ray charles in the 2004 biographical film ray


In [225]:
df["claim"] = df["claim"].apply(preprocess_claim)
df["evidence"] = df["evidence"].apply(preprocess_evidence)
df.head()

,claim,evidence,id,label,split
0,chris hemsworth appeared in a perfect getaway,hemsworth has also appeared in the science fic...,3,SUPPORTS,train
1,roald dahl is a writer,roald dahl 13 september 1916 23 november 1990 ...,7,SUPPORTS,train
2,roald dahl is a governor,roald dahl 13 september 1916 23 november 1990 ...,8,REFUTES,train
3,ireland has relatively low-lying mountains,the island s geography comprises relatively lo...,9,SUPPORTS,train
4,ireland does not have relatively low-lying mou...,the island s geography comprises relatively lo...,10,REFUTES,train


In [265]:
print(df["claim"].str.extract(r"\b(~)\b").any())
print(df["evidence"].str.extract(r"\b(~)\b").any())

0    False
dtype: bool
0    False
dtype: bool


In [266]:
PADDING_TOKEN = "~"

In [268]:
Vocabulary = namedtuple("Vocabulary", ["to_string", "from_string", "string"])

In [284]:
def build_vocabulary(tokens, padding_token):
    """
    Given a list of strings, builds the corresponding Vocabulary object
    """
    assert padding_token not in tokens
    words = sorted(set(tokens))
    vocabulary, inverse_vocabulary = dict(), dict()
    vocabulary[0] = str(padding_token)
    inverse_vocabulary[str(padding_token)] = 0
    for i, w in tqdm(enumerate(words)):
        vocabulary[i + 1] = w
        inverse_vocabulary[w] = i + 1
    return Vocabulary(
        to_string=vocabulary,
        from_string=inverse_vocabulary,
        string=[padding_token] + words,
    )

In [281]:
claim_tokens = list(flatten(df["claim"].str.split().tolist()))
evidence_tokens = list(flatten(df["evidence"].str.split().tolist()))
tokens = claim_tokens + evidence_tokens
tokens[:10]

['chris',
 'hemsworth',
 'appeared',
 'in',
 'a',
 'perfect',
 'getaway',
 'roald',
 'dahl',
 'is']

In [285]:
vocabulary = build_vocabulary(tokens, PADDING_TOKEN)

In [290]:
vocabulary.from_string["chris"]

9476

In [287]:
len(vocabulary.string)

43508

In [334]:
def to_indexes(values, to_index):
    """
    Given a list of keys and a dictionary indexed by those keys,
    return the corresponding values in the dictionary
    """
    return [to_index[v] for v in values]


df["indexed_claim"] = df["claim"].apply(
    lambda s: to_indexes(s.split(), vocabulary.from_string)
)
df["indexed_evidence"] = df["evidence"].apply(
    lambda s: to_indexes(s.split(), vocabulary.from_string)
)
df["indexed_label"] = pd.Categorical(df["label"], ordered=True).codes
df.head()

,claim,evidence,id,label,split,indexed_claim,indexed_evidence,indexed_label
0,chris hemsworth appeared in a perfect getaway,hemsworth has also appeared in the science fic...,3,SUPPORTS,train,"[9476, 19069, 4942, 20352, 3069, 29751, 17367]","[19069, 18742, 4240, 4942, 20352, 38680, 34479...",1
1,roald dahl is a writer,roald dahl 13 september 1916 23 november 1990 ...,7,SUPPORTS,train,"[33347, 11731, 21088, 3069, 42418]","[33347, 11731, 893, 35007, 1558, 2058, 28017, ...",1
2,roald dahl is a governor,roald dahl 13 september 1916 23 november 1990 ...,8,REFUTES,train,"[33347, 11731, 21088, 3069, 17844]","[33347, 11731, 893, 35007, 1558, 2058, 28017, ...",0
3,ireland has relatively low-lying mountains,the island s geography comprises relatively lo...,9,SUPPORTS,train,"[21040, 18742, 32610, 24122, 26679]","[38680, 21120, 33900, 17284, 10467, 32610, 241...",1
4,ireland does not have relatively low-lying mou...,the island s geography comprises relatively lo...,10,REFUTES,train,"[21040, 13184, 27962, 18802, 32610, 24122, 26679]","[38680, 21120, 33900, 17284, 10467, 32610, 241...",0


In [335]:
class FeverDataset(Dataset):
    """
    Fever dataset for fact checking
    """

    def __init__(self, df):
        self.df = df.copy()
        self.df = self.df.reset_index(drop=True)

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        assert isinstance(index, int)
        claim = self.df.loc[index, "indexed_claim"]
        evidence = self.df.loc[index, "indexed_evidence"]
        label = self.df.loc[index, "indexed_label"]
        return claim, evidence, label

In [336]:
train_dataset = FeverDataset(df[df["split"] == "train"])
val_dataset = FeverDataset(df[df["split"] == "val"])
test_dataset = FeverDataset(df[df["split"] == "test"])

In [337]:
def pad_batch(batch, padding_index):
    """
    This function expects to receive a list of tuples (i.e. a batch),
    s.t. each tuple contains one (claim, evidence, label) triple
    and returns the same sequences padded with the padding token
    """
    (claims, evidences, labels) = zip(*batch)
    claims_lenghts = [len(x) for x in claims]
    evidences_lenghts = [len(y) for y in evidences]
    padded_claims = pad_sequence(
        [torch.tensor(t) for t in claims],
        batch_first=True,
        padding_value=padding_index,
    )
    padded_evidences = pad_sequence(
        [torch.tensor(t) for t in evidences],
        batch_first=True,
        padding_value=padding_index,
    )
    labels_tensor = torch.tensor(labels, dtype=torch.bool)
    return padded_claims, padded_evidences, claims_lenghts, evidences_lenghts, labels

In [338]:
BATCH_SIZE = 8

In [339]:
default_dataloader = partial(
    DataLoader,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=partial(pad_batch, padding_index=vocabulary.from_string[PADDING_TOKEN]),
    pin_memory=True,
)

In [340]:
train_dataloader = default_dataloader(train_dataset)
val_dataloader = default_dataloader(val_dataset)
test_dataloader = default_dataloader(test_dataset)

In [312]:
EMBEDDING_DIMENSION = 300

In [313]:
embedding_model = utils.load_embedding_model(
    "glove", embedding_dimension=EMBEDDING_DIMENSION
)

In [325]:
def build_embedding_matrices(
    df, embedding_model, embedding_dimension, vocabulary, splits, method="normal",
):
    """
    Builds the embedding matrices (one for each split) of a specific dataset,
    given a pre-trained Gensim word embedding model
    """
    assert method in ("uniform", "normal")

    def uniform_embedding(interval=(-1, 1)):
        return interval[0] + np.random.sample(embedding_dimension) + interval[1]

    def normal_embedding():
        return np.random.normal(embedding_dimension)

    oov_terms = dict()
    embedding_matrices = dict.fromkeys(
        splits, np.zeros((len(vocabulary.string), embedding_dimension))
    )
    for split in splits:
        tokens = set(
            list(flatten(df[df["split"] == split]["claim"].str.split().tolist()))
            + list(flatten(df[df["split"] == split]["evidence"].str.split().tolist()))
        )
        for word in tokens:
            word_index = vocabulary.from_string[word]
            word_vector = np.zeros((1, embedding_dimension))
            # Words that are no OOV are taken from the Gensim model
            if word in embedding_model.vocab:
                word_vector = embedding_model[word]
            # Compute OOV embedding, if not already done
            elif word not in oov_terms:
                # OOV words computed as random normal vectors
                if method == "normal":
                    word_vector = normal_embedding()
                # OOV words computed as uniform vectors in range [-1, 1]
                elif method == "uniform":
                    word_vector = uniform_embedding()
                # The word is not OOV anymore
                oov_terms[word] = word_vector
            else:
                word_vector = oov_terms[word]
            embedding_matrices[split][word_index, :] = word_vector
    return embedding_matrices

In [326]:
embedding_matrices = build_embedding_matrices(
    df,
    embedding_model,
    EMBEDDING_DIMENSION,
    vocabulary,
    ["train", "val", "test"],
    method="normal",
)

In [328]:
embedding_matrices["train"][0, :]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

## Model definition

In [ ]:
class FactCheckingModel(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        lr=1e-3,
        padding_index=0,
        sentence_strategy="bov",
        merging_strategy="mean",
    ):
        """
        Build a generic fact checking model, with recurrent modules
        """
        super().__init__()

        # Padding
        self.padding_index = padding_index

        # Get device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Embedding module
        self.embedding = None

        # Strategy to perform sentence embedding
        assert sentence_strategy in ("bov", "mlp", "rnn_out_avg", "rnn_last")
        self.sentence_strategy = sentence_strategy

        # Strategy to merge claims and evidences
        assert merging_strategy in ("mean", "sum", "cat")
        self.merging_strategy = merging_strategy

        # Dense and dropout layers
        self.classifier = nn.Linear(
            embedding_dimension * 2
            if merging_strategy == "cat"
            else embedding_dimension,
            1,
        )

        # Criterion
        self.criterion = nn.BCEWithLogitsLoss().to(self.device)

        # Set default optimizer
        self.lr = lr
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

        # Define learning rate decay (defaults to no decay)
        self.lr_scheduler = None

        # Transfer model to device
        self.to(self.device)

    def lr_step(self):
        """
        Decay the learning rate, based on the defined schedule
        """
        if self.lr_scheduler is not None:
            self.lr_scheduler.step()

    def inject_embeddings(self, embeddings):
        """
        Inject the given embedding matrix in the Embedding module
        and make it non-trainable
        """
        self.embedding = nn.Embedding(
            embeddings.shape[0], embeddings.shape[1], padding_idx=self.padding_index
        )
        self.embedding.weight = nn.Parameter(torch.FloatTensor(embeddings))
        self.embedding.weight.requires_grad = False

    def count_parameters(self):
        """
        Return the total number of trainable parameters in the model
        """
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def init_weights(self):
        """
        Initialize all the model weights by sampling from a normal
        distribution with zero mean and small standard deviation
        """
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def _bag_of_vectors(self, claims, evidences):
        return claims.mean(dim=1), evidences.mean(dim=1)

    def _merge_embeddings(self, claims, evidences):
        if self.merging_strategy == "mean":
            return (claims + evidences) / 2
        elif self.merging_strategy == "sum":
            return claims + evidences
        elif self.merging_strategy == "cat":
            return torch.stack([claims, evidences], dim=1)
        return None

    def forward(self, claims, claims_lenghts, evidences, evidences_lenghts):
        """
        Perform a forward pass and return predictions over
        a mini-batch of sequences of the same lenght
        (one value for each possible tag for each input token)
        """
        assert self.embedding is not None
        embedded_claims = self.embedding(claims)
        embedded_evidences = self.embedding(evidences)
        packed_claims = pack_padded_sequence(
            embedded_claims, claims_lenghts, batch_first=True, enforce_sorted=False
        )
        packed_evidences = pack_padded_sequence(
            embedded_evidences,
            evidences_lenghts,
            batch_first=True,
            enforce_sorted=False,
        )
        bov_claims, bov_evidences = self._bag_of_vectors(
            packed_claims, packed_evidences
        )
        merged_inputs = self._merge_embeddings(bov_claims, bov_evidences)
        predictions = self.classifier(merged_inputs)
        return predictions

    def predict(self, predictions, labels):
        """
        Transform emission scores into tags indexes and return
        flattened Python lists containing predictions and ground truths
        """
        non_pad_elements = torch.where(tags.view(-1) != self.padding_index)[0]
        ground_truth = tags.view(-1)[non_pad_elements].detach().cpu().tolist()
        if not self.crf:
            predictions = predictions.view(-1, predictions.shape[-1])
            max_predictions = predictions.argmax(dim=1)
            return (
                max_predictions[non_pad_elements].detach().cpu().tolist(),
                ground_truth,
            )

        mask = torch.ones(tags.shape, dtype=torch.uint8)
        mask[tags == self.padding_index] = 0
        max_predictions = self.criterion.decode(predictions, mask=mask)
        return (list(flatten(max_predictions)), ground_truth)

    def loss(self, predictions, labels):
        """
        Compute the loss and return its value
        """
        return self.criterion(predictions, labels)

## Training

## Evaluation

## Comments/summary